## Eulerian Fluid Simulation
*still sensors that never moves*
- Eulerian representation uses still sensors in space, usually arranged in a
 regular grid/triangular mesh.
- A bit of math - but not too much.

### Overview
#### Material Derivatives: Lagrangian v.s. Eulerian (材料导数，物质导数)

\begin{equation*}
\frac{D}{Dt} := \frac{\partial}{\partial t}+{\rm u}\cdot\nabla
\end{equation*}

E.g.,<br>
\begin{equation*}\begin{array}{rcl}
\displaystyle\frac{DT}{Dt} & = & \displaystyle\frac{\partial T}{\partial t}+{\rm u}\cdot\nabla T \\
\displaystyle\frac{D{\rm u}_{x}}{Dt} & = & \displaystyle\frac{\partial {\rm u}_{x}}{\partial t}+{\rm u}\cdot\nabla {\rm u}_{x}
\end{array}\end{equation*}<br>
${\rm u}$: material (fluid) velocity. Many other names: Advective/Lagrangian/particle derivative.
Intuitively, change of physical quantity on a piece of material =
1. change due to time $\frac{\partial}{\partial t}$ (Eulerian).
2. change due to material movement ${\rm u}\cdot\nabla$.

#### (Incompressible) Navier-Stokes equations
\begin{array}{rcl}
\rho\displaystyle\frac{D{\rm u}}{Dt}&=&-\nabla p+\mu\nabla^{2}{\rm u}+\rho g \\
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+\nu\nabla^{2}{\rm u}+g \\
\nabla \cdot {\rm u}&=&0
\end{array}<br>
$\mu$: dynamic viscosity; $\nu=\frac{\mu}{\rho}$: kinematic viscosity.<br>
In graphics we usually drop **viscosity** except for highly viscous materials (e.g., honey)

#### Operator splitting
\begin{array}{rclc}
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+g,&\nabla \cdot {\rm u}=0
\end{array}<br>
Split the equations above into three parts:<br>
\begin{array}{rcllr}
\displaystyle\frac{D{\rm u}}{Dt}&=&0,\displaystyle\frac{D\alpha}{Dt}=0&(\mathbf{advection})&(1) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&g&({\rm external forces, optional})&(2) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot {\rm u}=0&(\mathbf{projection})&(3)
\end{array}<br>
$\alpha$: any physical property (temperature, color, smoke density etc.)

#### Eulerian ﬂuid simulation cycle
Time discretization with splitting: for each time step,<br>
1. Advection: “move” the ﬂuid feld. Solve ${\rm u}^{*}$ using  ${\rm u}^{t}$<br>
\begin{equation}
\displaystyle\frac{D{\rm u}}{Dt}=0,\displaystyle\frac{D\alpha}{Dt}=0
\end{equation}
2. External forces (optional): evaluate ${\rm u}^{**}$ using ${\rm u}^{*}$<br>
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=g
\end{equation}
3. Projection: make velocity field ${\rm u}^{t+1}$ dicergence-free based on ${\rm u}^{**}$<br>
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot{\rm u}^{t+1}=0
\end{equation}

### Grid
#### Spatial discretization using cell-centered grids
![fig](./cell-centered-grigs.png)<br>
Figure: ${\rm u}_{x},{\rm u}_{y},p$ are all stored at the center (orange) of cells

```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Spatial discretization using staggered grids
![fig](./staggered-grids.png)<br>
Figure: Red: ${\rm u}_{x}$; Green: ${\rm u}_{y}$; Blue: $p$.
```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n+1, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m+1)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Bilinear interpolation (双线性插值)
See PDF.

### Advection (对流)
#### Advection schemes
A trade-oﬀ between numerical viscosity, stability, performance and complexity:
- Semi-Lagrangian advection
- MacCormack/BFECC
- BiMocq
- Particle advection (PIC/FLIP/APIC/PolyPIC, later in this course)
- ...

#### Semi-Lagrangian advection
![Semi-Lagrangian](./semi-lagrangian.png)<br>
Figure: What should be the field value at **p** now based on the field and velocity
 at the previous time step? Well, just let reverse the simulation...<br>
 对x做semi-lagrangian，将结果存在new_x的代码为：
```python
import taichi as ti
@ti.func
def semi_lagrangian(x, new_x , dt):
    for I in ti.grouped(x):
        new_x[I] = sample_bilinear(x, backtrace(I, dt))
```
`backtrace(I, dt)`: 求I在dt段时间前的历史位置
##### what if ...
![what if](./what-if.png)<br>
Figure: The real trajectory of material parcels can be complex... Red: a naive estimation
of last position; Light gree: the true previous position.

In [11]:
import taichi as ti

ti.init(arch=ti.gpu)

use_mc = True
mc_clipping = True
pause = False

# Runge-Kutta order
rk = 3

n = 512
x = ti.field(ti.f32, shape=(n, n))
new_x = ti.field(ti.f32, shape=(n, n))
new_x_aux = ti.field(ti.f32, shape=(n, n))
dx = 1 / n
inv_dx = 1 / dx
dt = 0.05

stagger = ti.Vector([0.5, 0.5])


@ti.func
def Vector2(x, y):
    return ti.Vector([x, y])


@ti.func
def inside(p, c, r):
    return (p - c).norm_sqr() <= r * r


@ti.func
def inside_taichi(p):
    p = Vector2(0.5, 0.5) + (p - Vector2(0.5, 0.5)) * 1.2
    ret = -1
    if not inside(p, Vector2(0.50, 0.50), 0.55):
        if ret == -1:
            ret = 0
    if not inside(p, Vector2(0.50, 0.50), 0.50):
        if ret == -1:
            ret = 1
    if inside(p, Vector2(0.50, 0.25), 0.09):
        if ret == -1:
            ret = 1
    if inside(p, Vector2(0.50, 0.75), 0.09):
        if ret == -1:
            ret = 0
    if inside(p, Vector2(0.50, 0.25), 0.25):
        if ret == -1:
            ret = 0
    if inside(p, Vector2(0.50, 0.75), 0.25):
        if ret == -1:
            ret = 1
    if p[0] < 0.5:
        if ret == -1:
            ret = 1
    else:
        if ret == -1:
            ret = 0
    return ret


@ti.kernel
def paint():
    for i, j in ti.ndrange(n * 4, n * 4):
        ret = 1 - inside_taichi(Vector2(i / n / 4, j / n / 4))
        x[i // 4, j // 4] += ret / 16


@ti.func
def velocity(p):
    return ti.Vector([p[1] - 0.5, 0.5 - p[0]])


@ti.func
def vec(x, y):
    return ti.Vector([x, y])


@ti.func
def clamp(p):
    for d in ti.static(range(p.n)):
        p[d] = min(1 - 1e-4 - dx + stagger[d] * dx, max(p[d], stagger[d] * dx))
    return p


@ti.func
def sample_bilinear(x, p):
    p = clamp(p)

    p_grid = p * inv_dx - stagger

    I = ti.cast(ti.floor(p_grid), ti.i32)
    f = p_grid - I
    g = 1 - f

    return x[I] * (g[0] * g[1]) + x[I + vec(1, 0)] * (
            f[0] * g[1]) + x[I + vec(0, 1)] * (
                   g[0] * f[1]) + x[I + vec(1, 1)] * (f[0] * f[1])


@ti.func
def sample_min(x, p):
    p = clamp(p)
    p_grid = p * inv_dx - stagger
    I = ti.cast(ti.floor(p_grid), ti.i32)

    return min(x[I], x[I + vec(1, 0)], x[I + vec(0, 1)], x[I + vec(1, 1)])


@ti.func
def sample_max(x, p):
    p = clamp(p)
    p_grid = p * inv_dx - stagger
    I = ti.cast(ti.floor(p_grid), ti.i32)

    return max(x[I], x[I + vec(1, 0)], x[I + vec(0, 1)], x[I + vec(1, 1)])


@ti.func
def backtrace(I, dt):
    p = (I + stagger) * dx
    if ti.static(rk == 1):
        p -= dt * velocity(p)
    elif ti.static(rk == 2):
        p_mid = p - 0.5 * dt * velocity(p)
        p -= dt * velocity(p_mid)
    elif ti.static(rk == 3):
        v1 = velocity(p)
        p1 = p - 0.5 * dt * v1
        v2 = velocity(p1)
        p2 = p - 0.75 * dt * v2
        v3 = velocity(p2)
        p -= dt * (2 / 9 * v1 + 1 / 3 * v2 + 4 / 9 * v3)
    else:
        ti.static_print(f"RK{rk} is not supported.")

    return p


@ti.func
def semi_lagrangian(x, new_x, dt):
    # Note: this loop is parallelized
    for I in ti.grouped(x):
        new_x[I] = sample_bilinear(x, backtrace(I, dt))


# Reference: https://github.com/ziyinq/Bimocq/blob/master/src/bimocq2D/BimocqSolver2D.cpp

@ti.func
def maccormack(x, dt):
    semi_lagrangian(x, new_x, dt)
    semi_lagrangian(new_x, new_x_aux, -dt)

    for I in ti.grouped(x):
        new_x[I] = new_x[I] + 0.5 * (x[I] - new_x_aux[I])

        if ti.static(mc_clipping):
            source_pos = backtrace(I, dt)
            min_val = sample_min(x, source_pos)
            max_val = sample_max(x, source_pos)

            if new_x[I] < min_val or new_x[I] > max_val:
                new_x[I] = sample_bilinear(x, source_pos)


@ti.kernel
def advect():
    if ti.static(use_mc):
        maccormack(x, dt)
    else:
        semi_lagrangian(x, new_x, dt)

    for I in ti.grouped(x):
        x[I] = new_x[I]


paint()

gui = ti.GUI('Advection schemes', (512, 512))

while True:
    for e in gui.get_events():
        if e.key == ti.GUI.EXIT:
            break
        elif e.key == ti.GUI.PRESS:
            pass
    if not gui.running:
        break

    while gui.get_event(ti.GUI.PRESS):
        if gui.event.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]: exit(0)
        if gui.event.key == ti.GUI.SPACE:
            pause = not pause
    if not pause:
        for i in range(1):
            advect()
    gui.set_image(x.to_numpy())
    gui.show()

[Taichi] Starting on arch=cuda
[Taichi] materializing...


由于旋转时每次做位置估计都会产生一定误差，导致图案越来越下。（Forward Euler）的弊端
#### Going back in time (Demo)
Initial value problem (ODE): simply use explicit time integration schemes, e.g.,<br>
1. Forward Euler ("RK1")<br>
`p -= dt * velocity(p)`
2. Explicit Midpoint("RK2")<br>
```
p_mid = p - 0.5 * dt * velocity(p)
p -= dt * velocity(p_mid)
```
3. RK3<br>
```
v1 = velocity(p)
p1 = p - 0.5 * dt * v1
v2 = velocity(p1)
p2 = p - 0.75 * dt * v2
v3 = velocity(p2)
p -= dt * (2 / 9 * v1 + 1 / 3 * v2 + 4 / 9 * v3)
```

#### BFECC and MacCormack advection schemes
BFECC: Back and Forth Error Compensation and Correction
- $\mathbf{x}^{*}={\rm SL}(\mathbf{x}, \Delta t)$
- $\mathbf{x}^{**}={\rm SL}(\mathbf{x}^{*}, -\Delta t)$
- Estimate the error $\mathbf{x^{error}}=\frac{1}{2}(\mathbf{x}^{**}-\mathbf{x})$
- Apply the error $\mathbf{x^{final}}=\mathbf{x}^{*}+\mathbf{x^{error}}$

Be careful: need to prevent overshooting.<br>
先回到过去，得到一个$\mathbf{x}^{*}$；然后根据$\mathbf{x}^{*}$预测未来，
得到另一个$$\mathbf{x}^{**}$$。如果SL是准确的，那么应当有
$\mathbf{x}^{*}==\mathbf{x}^{**}$

In [7]:
import taichi as ti
import numpy as np
import math

ti.init(arch=ti.gpu)

eps = 0.01
dt = 0.1

n_vortex = 4
n_tracer = 200000

pos = ti.Vector.field(2, ti.f32, shape=n_vortex)
new_pos = ti.Vector.field(2, ti.f32, shape=n_vortex)
vort = ti.field(ti.f32, shape=n_vortex)

tracer = ti.Vector.field(2, ti.f32, shape=n_tracer)


@ti.func
def compute_u_single(p, i):
    r2 = (p - pos[i]).norm() ** 2
    uv = ti.Vector([pos[i].y - p.y, p.x - pos[i].x])
    return vort[i] * uv / (r2 * math.pi) * 0.5 * (1.0 - ti.exp(-r2 / eps ** 2))


@ti.func
def compute_u_full(p):
    u = ti.Vector([0.0, 0.0])
    for i in range(n_vortex):
        u += compute_u_single(p, i)
    return u


@ti.kernel
def integrate_vortex():
    for i in range(n_vortex):
        v = ti.Vector([0.0, 0.0])
        for j in range(n_vortex):
            if i != j:
                v += compute_u_single(pos[i], j)
        new_pos[i] = pos[i] + dt * v

    for i in range(n_vortex):
        pos[i] = new_pos[i]


@ti.kernel
def advect():
    for i in range(n_tracer):
        # Ralston's third-order method
        p = tracer[i]
        v1 = compute_u_full(p)
        v2 = compute_u_full(p + v1 * dt * 0.5)
        v3 = compute_u_full(p + v2 * dt * 0.75)
        tracer[i] += (2 / 9 * v1 + 1 / 3 * v2 + 4 / 9 * v3) * dt


pos[0] = [0, 1]
pos[1] = [0, -1]
pos[2] = [0, 0.3]
pos[3] = [0, -0.3]
vort[0] = 1
vort[1] = -1
vort[2] = 1
vort[3] = -1


@ti.kernel
def init_tracers():
    for i in range(n_tracer):
        tracer[i] = [ti.random() - 0.5, ti.random() * 3 - 1.5]


init_tracers()

gui = ti.GUI("Vortex", (1024, 512), background_color=0xFFFFFF)

for T in range(1000):
    for e in gui.get_events():
        if e.key == ti.GUI.EXIT:
            break
        elif e.key == ti.GUI.PRESS:
            pass
    if not gui.running:
        break

    for i in range(4):  # substeps
        advect()
        integrate_vortex()

    gui.circles(
        tracer.to_numpy() * np.array([[0.05, 0.1]]) + np.array([[0.0, 0.5]]),
        radius=0.5,
        color=0x0)

    gui.show()


[Taichi] Starting on arch=cuda
[Taichi] materializing...
